# Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
import matlab.engine as engi
import matlab as mat
import math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy.io import loadmat
from sklearn.cluster import SpectralClustering
from sklearn.metrics import f1_score
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import metrices

# Start matlab service

In [2]:
eng = engi.start_matlab()
eng.addpath(r'matlab_CTKCCA/',nargout=0)
eng.addpath(r'matlab_KS/',nargout=0)

# variables

In [3]:
source_data_path = 'data/Relink/Apache/Apache.csv'
target_data_path = 'data/AEEEM/EQ/EQ.csv'
# source_data_path = 'data/1385/converted/ivef-sdk.csv'
# target_data_path = 'data/1385/converted/adaptagrams.csv'

result_path = 'result/result.csv'
repeats = 20
ratio = 0.1
lrank = 70
reg = 1E-5

# Data loading and Normalizing Data

In [4]:
def load_data(path):
    df = pd.read_csv(path)
    df_columns = df.columns
    _temp = []
    for col in df_columns:
        col = col.replace('$','')
        col = col.replace('>','')
        col = col.replace('<','')
        _temp.append(col)
    df.columns = _temp
    if "Defective" in _temp:
        d = {'T': True, 'F': False}
        df["Defective"] = df["Defective"].map(d)
    return df

In [5]:
source_df = load_data(source_data_path)
target_df = load_data(target_data_path)

# decomposing kernel correlation metrics

## Matlab integration - CTKCCA

In [6]:
def transform_data_CTKCCA(source_df,target_df):
    mat_source_df = mat.double(source_df.values.T.tolist())
    mat_target_df = mat.double(target_df.values.T.tolist())
    X = eng.CTKCCA(mat_source_df,mat_target_df,nargout=4)
    train_X,train_y = np.array(X[0]),np.array(X[1]).tolist()[0]
    test_X,test_y = np.array(X[2]),np.array(X[3]).tolist()[0]
    return train_X,train_y,test_X,test_y
train_X,train_y,test_X,test_y = transform_data_CTKCCA(source_df,target_df)

In [7]:
clf = LogisticRegression()
clf.fit(train_X,train_y)
predicted = clf.predict(test_X)
print(classification_report(test_y, predicted))

              precision    recall  f1-score   support

         0.0       0.77      1.00      0.87       195
         1.0       1.00      0.54      0.70       129

    accuracy                           0.82       324
   macro avg       0.88      0.77      0.79       324
weighted avg       0.86      0.82      0.80       324



## Matlab integration - KS
### may need to include a feature selector for source project

In [8]:
def transform_data_KS(source_df,target_df):
    mat_source_df = mat.double(source_df.values.T.tolist())
    mat_target_df = mat.double(target_df.values.T.tolist())
    X = eng.HDP_KS(mat_source_df,mat_target_df,nargout=4)
    train_X,train_y = np.array(X[0]),np.array(X[1]).tolist()[0]
    test_X,test_y = np.array(X[2]),np.array(X[3]).tolist()[0]
    return train_X,train_y,test_X,test_y
train_X,train_y,test_X,test_y = transform_data_KS(source_df,target_df)

In [9]:
clf = LogisticRegression()
clf.fit(train_X,train_y)
predicted = clf.predict(test_X)
print(classification_report(test_y, predicted))

              precision    recall  f1-score   support

         0.0       0.70      0.73      0.72       195
         1.0       0.57      0.53      0.55       129

    accuracy                           0.65       324
   macro avg       0.63      0.63      0.63       324
weighted avg       0.65      0.65      0.65       324



# Run Exp with clustering

In [10]:
results_dist = pd.read_csv('result/MMD_full.csv',index_col=0)
df_MMD_updated = results_dist.abs()
df_MMD_updated.values[[np.arange(df_MMD_updated.shape[0])]*2] = 0
df_MMD_updated = df_MMD_updated[0:50]
df_MMD_updated = df_MMD_updated[df_MMD_updated.columns[0:50]]
clustering = SpectralClustering(n_clusters=3,affinity='precomputed').fit(df_MMD_updated)
clusters = zip(df_MMD_updated.index.tolist(),clustering.labels_.tolist())
cluster_projects = {}
for cluster in clusters:
    if cluster[1] not in cluster_projects.keys():
        cluster_projects[cluster[1]] = []
    cluster_projects[cluster[1]].append(cluster[0])

/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/manifold/_spectral_embedding.py:212: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [15]:
score_dict = {}
f1_score_dict = {}
precision_score_dict = {}
recall_score_dict = {}
pf_score_dict = {}
for cluster in cluster_projects:
    projects = cluster_projects[cluster]
    if 'precision'  not in score_dict.keys():
        score_dict['precision'] = {}
        score_dict['recall'] = {}
        score_dict['pf'] = {}
        score_dict['f1'] = {}
    if cluster not in f1_score_dict.keys():
        score_dict['precision'][cluster] = {}
        score_dict['recall'][cluster] = {}
        score_dict['pf'][cluster] = {}
        score_dict['f1'][cluster] = {}
        f1_score_dict[cluster] = {}
        precision_score_dict[cluster] = {}
        recall_score_dict[cluster] = {}
        pf_score_dict[cluster] = {}
    for s_project in projects:
        print(s_project)
        if s_project not in f1_score_dict.keys():
            score_dict['precision'][cluster][s_project] = {}
            score_dict['recall'][cluster][s_project] = {}
            score_dict['pf'][cluster][s_project] = {}
            score_dict['f1'][cluster][s_project] = {}
            f1_score_dict[cluster][s_project] = {}
            precision_score_dict[cluster][s_project] = {}
            recall_score_dict[cluster][s_project] = {}
            pf_score_dict[cluster][s_project] = {}
        for d_project in projects:
#             print(s_project,d_project),
            if s_project == d_project:
                continue
            source_df = load_data('data/All/' + s_project)
            target_df = load_data('data/All/' + d_project)
            train_X,train_y,test_X,test_y = transform_data_CTKCCA(source_df,target_df)
            clf = LogisticRegression()
            clf.fit(train_X,train_y)
            predicted = clf.predict(test_X)
            abcd = metrices.measures(test_y,predicted)
            f1 = f1_score(test_y, predicted, average='binary')
            recall = abcd.calculate_recall()
            precision = abcd.calculate_precision()
            pf = abcd.get_pf()
            score_dict['precision'][cluster][s_project][d_project] = precision
            score_dict['recall'][cluster][s_project][d_project] = recall
            score_dict['pf'][cluster][s_project][d_project] = pf
            score_dict['f1'][cluster][s_project][d_project] = f1
            f1_score_dict[cluster][s_project][d_project] = f1
            precision_score_dict[cluster][s_project][d_project] = precision
            recall_score_dict[cluster][s_project][d_project] = recall
            pf_score_dict[cluster][s_project][d_project] = pf
with open('result/f1_clsutered_self.pkl', 'wb') as handle:
    pickle.dump(f1_score_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Apache.csv
EQ.csv


/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmaju

Safe.csv
Zxing.csv


/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmaju

ant-1.6.csv


...

/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmaju

mc1.csv


/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmaju

poi-1.5.csv


/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suvodeepmaju

In [ ]:
with open('result/f1_clsutered.pkl', 'rb') as handle:
    f1_score_dict = pickle.load(handle)
cluster_results = pd.Series()
for key in f1_score_dict.keys():
    f1_score_df = pd.DataFrame.from_dict(f1_score_dict[key],orient='index')
    print(f1_score_df.shape)
    f1_score_df = f1_score_df.reindex(sorted(f1_score_df.columns), axis=0)
    f1_score_df = f1_score_df.reindex(sorted(f1_score_df.columns), axis=1)
#     print(f1_score_df.median(axis = 1))
#     print(f1_score_df.median(axis = 1).idxmax())
#     print(f1_score_df.loc[f1_score_df.median(axis = 1).idxmax()])
    row = f1_score_df.loc[f1_score_df.median(axis = 1).idxmax()]
    cluster_results = cluster_results.append(row)
print(cluster_results)

In [ ]:
f1_score_dict = {}
projects = df_MMD_updated.index.tolist()
for s_project in projects:
    print(s_project)
    if s_project not in f1_score_dict.keys():
        f1_score_dict[s_project] = {}
    for d_project in projects:
#             print(s_project,d_project),
        if s_project == d_project:
            continue
        source_df = load_data('data/All/' + s_project)
        target_df = load_data('data/All/' + d_project)
        train_X,train_y,test_X,test_y = transform_data_CTKCCA(source_df,target_df)
        clf = LogisticRegression()
        clf.fit(train_X,train_y)
        predicted = clf.predict(test_X)
        f1 = f1_score(test_y, predicted, average='binary')
        f1_score_dict[s_project][d_project] = f1

In [ ]:
f1_score_df = pd.DataFrame.from_dict(f1_score_dict,orient='index')
f1_score_df = f1_score_df.reindex(sorted(f1_score_df.columns), axis=0)
f1_score_df = f1_score_df.reindex(sorted(f1_score_df.columns), axis=1)
f1_score_df.to_csv('result/f1_all.csv')
#     print(f1_score_df.median(axis = 1))
print(f1_score_df.median(axis = 1).max())

In [ ]:
f1_score_df = pd.read_csv('result/f1_all.csv',index_col=0)
f1_score_df = f1_score_df.reindex(sorted(f1_score_df.columns), axis=0)
f1_score_df = f1_score_df.reindex(sorted(f1_score_df.columns), axis=1)
row = f1_score_df.loc[f1_score_df.median(axis = 1).idxmax()]

In [ ]:
result = []
for index in row.index:
    result.append([index,row[index],cluster_results[index]])

In [ ]:
result_df = pd.DataFrame(result,columns=['project','all','clustered'])
result_df.to_csv('result/compare.csv')

In [14]:
score_dict

{'precision': {2: {'Apache.csv': {'EQ.csv': {},
    'Safe.csv': {},
    'Zxing.csv': {},
    'ant-1.6.csv': {},
    'camel-1.2.csv': {},
    'ivy-2.0.csv': {},
    'jedit-3.2.csv': {},
    'jedit-4.1.csv': {},
    'log4j-1.2.csv': {},
    'lucene-2.0.csv': {},
    'lucene-2.2.csv': {}},
   'EQ.csv': {'Apache.csv': {},
    'Safe.csv': {},
    'Zxing.csv': {},
    'ant-1.6.csv': {},
    'camel-1.2.csv': {},
    'ivy-2.0.csv': {},
    'jedit-3.2.csv': {},
    'jedit-4.1.csv': {},
    'log4j-1.2.csv': {},
    'lucene-2.0.csv': {},
    'lucene-2.2.csv': {}},
   'Safe.csv': {'Apache.csv': {},
    'EQ.csv': {},
    'Zxing.csv': {},
    'ant-1.6.csv': {},
    'camel-1.2.csv': {},
    'ivy-2.0.csv': {},
    'jedit-3.2.csv': {},
    'jedit-4.1.csv': {},
    'log4j-1.2.csv': {},
    'lucene-2.0.csv': {},
    'lucene-2.2.csv': {}},
   'Zxing.csv': {'Apache.csv': {},
    'EQ.csv': {},
    'Safe.csv': {},
    'ant-1.6.csv': {},
    'camel-1.2.csv': {},
    'ivy-2.0.csv': {},
    'jedit-3.2.csv': {},